# Drosophila Melanogaster

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import Bio
import torch as torch
from torch.utils.data import Dataset, DataLoader
print("Biopython v" + Bio.__version__)
from timeit import default_timer as timer
import copy
import math
import sys




Biopython v1.78


In [2]:
from Bio import SeqIO
count = 0
sequences = [] # Here we are setting up an array to save our sequences for the next step

for seq_record in SeqIO.parse("./Genomedata/genome.fa", "fasta"):
    if (count < 6):
        sequences.append(seq_record)
        print("Id: " + seq_record.id + " \t " + "Length: " + str("{:,d}".format(len(seq_record))) )
        print(repr(seq_record.seq) + "\n")
        count = count + 1

sequences.pop()

Id: chr2L 	 Length: 23,513,712
Seq('Cgacaatgcacgacagaggaagcagaacagatatttagattgcctctcattttc...gag')

Id: chr2R 	 Length: 25,286,936
Seq('CTCAAGATAccttctacagattatttaaagctagtgcacaacaacaataaattg...ttc')

Id: chr3L 	 Length: 28,110,227
Seq('TAGGGAGAAATATGATCgcgtatgcgagagtagtgccaacatattgtgctcttt...tat')

Id: chr3R 	 Length: 32,079,331
Seq('acgggaccgagtatagtaccagtacgcgaccagtacgggagcagtacggaacca...ttc')

Id: chr4 	 Length: 1,348,131
Seq('ttattatattattatattattatattattatattattatattattatattatta...GAA')

Id: chrM 	 Length: 19,524
Seq('AATGAATTGCCTGATAAAAAGGATTACCTTGATAGGGTAAATCATGCAGTTTTC...ATT')



SeqRecord(seq=Seq('AATGAATTGCCTGATAAAAAGGATTACCTTGATAGGGTAAATCATGCAGTTTTC...ATT'), id='chrM', name='chrM', description='chrM', dbxrefs=[])

In [3]:
sequences

[SeqRecord(seq=Seq('Cgacaatgcacgacagaggaagcagaacagatatttagattgcctctcattttc...gag'), id='chr2L', name='chr2L', description='chr2L', dbxrefs=[]),
 SeqRecord(seq=Seq('CTCAAGATAccttctacagattatttaaagctagtgcacaacaacaataaattg...ttc'), id='chr2R', name='chr2R', description='chr2R', dbxrefs=[]),
 SeqRecord(seq=Seq('TAGGGAGAAATATGATCgcgtatgcgagagtagtgccaacatattgtgctcttt...tat'), id='chr3L', name='chr3L', description='chr3L', dbxrefs=[]),
 SeqRecord(seq=Seq('acgggaccgagtatagtaccagtacgcgaccagtacgggagcagtacggaacca...ttc'), id='chr3R', name='chr3R', description='chr3R', dbxrefs=[]),
 SeqRecord(seq=Seq('ttattatattattatattattatattattatattattatattattatattatta...GAA'), id='chr4', name='chr4', description='chr4', dbxrefs=[])]

In [4]:
agsts = pd.read_csv('./Genomedata/genes-augustus.csv')

In [5]:
agsts['chrom'].unique()

array(['chr2L', 'chr2R', 'chr3L', 'chr3R', 'chr4', 'chrUn_CP007080v1',
       'chrUn_CP007081v1', 'chrUn_CP007097v1', 'chrUn_DS483755v1',
       'chrUn_DS483780v1', 'chrUn_DS483783v1', 'chrUn_DS483801v1',
       'chrUn_DS483878v1', 'chrUn_DS484006v1', 'chrUn_DS484105v1',
       'chrUn_DS484134v1', 'chrUn_DS484375v1', 'chrUn_DS484521v1',
       'chrUn_DS484581v1', 'chrUn_DS484779v1', 'chrUn_DS485083v1',
       'chrUn_DS485107v1', 'chrUn_DS485610v1', 'chrX',
       'chrX_DS483995v1_random', 'chrY', 'chrY_CP007109v1_random',
       'chrY_CP007113v1_random', 'chrY_CP007116v1_random',
       'chrY_DS483742v1_random', 'chrY_DS483778v1_random',
       'chrY_DS483875v1_random', 'chrY_DS483931v1_random',
       'chrY_DS484390v1_random', 'chrY_DS484643v1_random',
       'chrY_DS485048v1_random', 'chrY_DS485159v1_random'], dtype=object)

Finding the sequences 

In [6]:
x = np.array([rec for rec in str(sequences[1].seq)], np.dtype('U'))


In [7]:
print(set(x))
print(np.count_nonzero(x == 'N'))

{'c', 'n', 'g', 'A', 'T', 'G', 'C', 'a', 'N', 't'}
6600


In [102]:
len(x)

23513712

In [10]:
str(x[0])

'C'

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
class SeqDataset(Dataset):

    def __init__(self, data):
        x = np.array([np.array(rec) for rec in str(data)], np.dtype('U'))
        self.X = x

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (X[idx])
seq1 = SeqDataset(sequences[0].seq)


In [20]:

train_loader = DataLoader(
    dataset=seq1, 
    batch_size=1, shuffle=False)

In [6]:
x = np.array([np.array(rec) for rec in str(sequences[0].seq)], np.dtype('U'))

In [27]:
x = torch.Tensor([[1,2,3,4]]).to(device)
y = torch.Tensor([[2],[3],[4],[5]]).to(device)

In [28]:
x * y

tensor([[ 2.,  4.,  6.,  8.],
        [ 3.,  6.,  9., 12.],
        [ 4.,  8., 12., 16.],
        [ 5., 10., 15., 20.]], device='cuda:0')

create Dataset

In [29]:
for S in sequences:
    agsts.loc[agsts['chrom'] == S.id]

,bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,585,g1.t1,chr2L,+,6485,10574,6590,9276,6,"6485,7573,7979,8192,8667,10454,","6808,7910,8116,8589,9741,10574,",0,g1,cmpl,cmpl,"0,2,0,2,0,-1,"
1,585,g1.t2,chr2L,+,6485,10574,6590,9276,5,"6485,7573,8192,8667,10454,","6808,8116,8589,9741,10574,",0,g1,cmpl,cmpl,"0,2,2,0,-1,"
2,585,g2.t1,chr2L,-,10706,40875,11770,40784,26,"10706,12285,13519,13682,14932,22268,22527,2274...","12221,12928,13625,14874,15275,22446,22687,2293...",0,g2,cmpl,cmpl,"2,1,0,2,1,0,2,1,1,0,0,2,2,0,1,2,2,1,2,2,0,2,2,..."
3,585,g3.t1,chr2L,+,56305,71879,67624,70895,6,"56305,61885,67568,67891,68084,70606,","56356,61966,67762,68023,70549,71879,",0,g3,cmpl,cmpl,"-1,-1,0,0,0,2,"
4,585,g3.t2,chr2L,+,56305,76202,67624,76098,11,"56305,66675,67568,67891,68084,70606,72597,7447...","56356,67003,67762,68023,70549,70806,72977,7457...",0,g3,cmpl,cmpl,"-1,-1,0,0,0,2,1,0,0,2,0,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,760,g2068.t2,chr2L,+,22982111,22992912,22982197,22991943,6,"22982111,22990753,22991055,22991207,22991394,2...","22982593,22990892,22991153,22991339,22991534,2...",0,g2068,cmpl,cmpl,"0,0,1,0,0,2,"
2524,760,g2069.t1,chr2L,-,22999565,23025931,22999672,23016070,9,"22999565,23003746,23003990,23005406,23012217,2...","23000966,23003933,23005346,23005686,23012403,2...",0,g2069,cmpl,cmpl,"2,1,1,0,0,0,0,0,-1,"
2525,11,g2069.t2,chr2L,-,22999585,23068851,22999672,23068217,11,"22999585,23003746,23003990,23005406,23012217,2...","23000966,23003933,23005346,23005686,23012403,2...",0,g2069,cmpl,cmpl,"2,1,1,0,0,0,0,1,0,0,-1,"
2526,761,g2070.t1,chr2L,-,23161126,23167576,23161957,23167499,4,"23161126,23162211,23162924,23167495,","23162139,23162872,23163058,23167576,",0,g2070,cmpl,cmpl,"1,0,1,0,"


In [16]:
sequences[0].seq[6480:6495].transcribe()

Seq('UGUAAAUUCCAAUUA')

In [19]:
sequences[0].seq[7566:7593].transcribe()


Seq('CUUUCAGCAACCGAGAAGAGAACCCAC')

In [20]:
sequences[0].seq[10570:10584].transcribe()

Seq('AAAAUGUGGGCACG')

In [8]:
i = s_id['name'][2][:-3] + '.t2'
print(s_id[s_id['name'].isin([i])].empty)


NameError: name 's_id' is not defined

In [122]:
['N'] * 2

['n', 'n']

In [123]:
ch = np.array(['N'] * s_id.iloc[-1]['txEnd'],dtype='U')


In [125]:
ch

array(['N', 'N', 'N', ..., 'N', 'N', 'N'], dtype='<U1')

In [124]:
len(ch)

23371726

In [49]:

s_id = agsts.loc[agsts['chrom'] == 'chr2L']
for index, row in s_id.iterrows():
    if row['name'][-3:] == '.t2':
        #Code exons
        
    check_t2 = row['name'][:-3] + '.t2'
    if s_id[s_id['name'].isin([check_t2])].empty:
        continue
    else: 
        #Code exons

IndentationError: expected an indented block (<ipython-input-49-dd615feab829>, line 5)

In [85]:
x= s_id.iloc[0]

In [86]:

ch[x['txStart']:x['txEnd']] = 'T'

In [70]:
ch[6600]

'T'

In [87]:
Estart = np.array(x['exonStarts'].split(',')[:-1]).astype(int)
Eend = np.array(x['exonEnds'].split(',')[:-1]).astype(int)

In [91]:
for i in range(len(Estart)):
    ch[Estart[i]:Eend[i]+1] = 'E'
    if i != len(Estart) - 1:
        ch[Eend[i]+1:Estart[i+1]] = 'I'

In [94]:
ch[6809]

'I'

In [131]:
seq = sequences[0].seq
arr = np.array([rec for rec in str(seq)],dtype='U')
a = np.array([x.lower() if isinstance(x, str) else x for x in arr])

In [132]:
a

array(['c', 'g', 'a', ..., 'g', 'a', 'g'], dtype='<U1')

In [161]:
s_id_aug = agsts.loc[agsts['chrom'] == 'chr2R']

In [172]:
s_id_aug.iloc[2705]

IndexError: single positional indexer is out-of-bounds

In [99]:
def code_Exon_Intron(coding_row, ch):
    ch[coding_row['txStart']:coding_row['txEnd']] = 'T'
    Estart = np.array(coding_row['exonStarts'].split(',')[:-1]).astype(int)
    Eend = np.array(coding_row['exonEnds'].split(',')[:-1]).astype(int)
    for i in range(len(Estart)):
        ch[Estart[i]:Eend[i]+1] = 'E'
        if i != len(Estart) - 1:
            ch[Eend[i]+1:Estart[i+1]] = 'I'
    return ch

def code_sequence(augustus, sequences):
    df = pd.DataFrame(columns=['id','x','y'])
    
    for seq in sequences:
        s_id_aug = augustus.loc[augustus['chrom'] == seq.id]

        y = np.array(['N'] * s_id_aug.iloc[-1]['txEnd'],dtype='U')

        t = np.array([rec for rec in str(seq.seq)],dtype='U')[:s_id_aug.iloc[-1]['txEnd']]
        x = np.array([x.upper() if isinstance(x, str) else x for x in t])


        for index, row in s_id_aug.iterrows():

            if row['name'][-3:] == '.t2':
                y = code_Exon_Intron(row, y)
                
            check_t2 = row['name'][:-3] + '.t2'
            if s_id_aug[s_id_aug['name'].isin([check_t2])].empty:
                continue
            else: 
                y = code_Exon_Intron(row, y)
        # print(x[0],'\n',x[1],'\n/',x[2],'\n',x[3],'\n',x[100])
        print(len(x.tolist()))
        char = pd.Series({'id': seq.id, 'x':x.tolist(), 'y':y.tolist()})
        print(char['x'][:20])
        df = df.append(char,ignore_index=True)
    return df

In [100]:
temp = code_sequence(agsts, sequences)


23371726
['C', 'G', 'A', 'C', 'A', 'A', 'T', 'G', 'C', 'A', 'C', 'G', 'A', 'C', 'A', 'G', 'A', 'G', 'G', 'A']
25254583
['C', 'T', 'C', 'A', 'A', 'G', 'A', 'T', 'A', 'C', 'C', 'T', 'T', 'C', 'T', 'A', 'C', 'A', 'G', 'A']
27866355
['T', 'A', 'G', 'G', 'G', 'A', 'G', 'A', 'A', 'A', 'T', 'A', 'T', 'G', 'A', 'T', 'C', 'G', 'C', 'G']
32070586
['A', 'C', 'G', 'G', 'G', 'A', 'C', 'C', 'G', 'A', 'G', 'T', 'A', 'T', 'A', 'G', 'T', 'A', 'C', 'C']
1271726
['T', 'T', 'A', 'T', 'T', 'A', 'T', 'A', 'T', 'T', 'A', 'T', 'T', 'A', 'T', 'A', 'T', 'T', 'A', 'T']


In [101]:
print(len(temp.iloc[0]['x']))
print(len(temp.iloc[0]['y']))
print(temp.iloc[0]['x'][:20])
print(temp.iloc[0]['y'][-1])

23371726
23371726
['C', 'G', 'A', 'C', 'A', 'A', 'T', 'G', 'C', 'A', 'C', 'G', 'A', 'C', 'A', 'G', 'A', 'G', 'G', 'A']
N


In [102]:
def prepare_segment(x,y):
    df = pd.DataFrame(columns = ['x','y'])
    last_i = 0
    for i in range(100,len(x),100):
        df = df.append({'x': x[last_i:i], 'y':['START'] + y[last_i:i] + ['STOP']}, ignore_index = True)
        last_i = i
    return df

In [103]:
for i in range(0,1004,100):
    print(i)

0
100
200
300
400
500
600
700
800
900
1000


In [104]:
df = prepare_segment(temp.iloc[4]['x'],temp.iloc[4]['y'])

In [111]:
df.iloc[6500]['y']

['START',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'STOP']

In [65]:
class MultiDimnDict():
    
    def __init__(self,axes):
        self.axes = axes

        self.data = {}
        prev = None

        for i in range(len(axes)-1, -1, -1):
            x = {}
            for n in axes[i]:
                if prev == None:
                    x[n] = 0
                else:
                    x[n] = copy.deepcopy(prev)
            prev = x
            self.data = x

    def __get_string(self, x,_str, depth):
        if type(list(x.values())[0]) == type(dict()):
            for i in x:
                _str += self.__get_string(x[i], depth * " " + i + "  \n", depth+1)
                # print(_str)
            return _str
        else:
            return _str + " " * depth +str(x) +"\n"

    def __str__(self):
        string = self.__get_string(self.data, "", 0)
        return string

    def add_value_helper(self,lst,val, temp):
        if len(lst) == 1:
            temp[lst[0]] += val
            return temp
        try :
            # print(lst[1:],val,temp[lst[0]])

            temp[lst[0]] = self.add_value_helper(lst[1:],val,temp[lst[0]])
            return temp
        except:
            return 0

    def add_value(self,lst,val):
        if len(lst) == len(self.axes):
            return self.add_value_helper(lst,val,self.data)
        else:
            return 0
    
    

x= MultiDimnDict([['N','I','E'],['N','I','E'],['A','B']])

print(x)


N  
 N  
  {'A': 0, 'B': 0}
 I  
  {'A': 0, 'B': 0}
 E  
  {'A': 0, 'B': 0}
I  
 N  
  {'A': 0, 'B': 0}
 I  
  {'A': 0, 'B': 0}
 E  
  {'A': 0, 'B': 0}
E  
 N  
  {'A': 0, 'B': 0}
 I  
  {'A': 0, 'B': 0}
 E  
  {'A': 0, 'B': 0}



Cant write to dataframe since the character array turns into shit

Steps

1. Create the count matrices
2. Use viterbi algo

In [120]:
class HMM():
    def __init__(self,data,vocab_x,vocab_y):
        self.data = data
        self.vocab_x = vocab_x
        self.vocab_y = vocab_y

    def calculate_emissions(self):
        emissions = MultiDimnDict([self.vocab_y, self.vocab_x, self.vocab_x])
        self.emissions_yi_xi = MultiDimnDict([self.vocab_y, self.vocab_x])
        count_yi = MultiDimnDict([self.vocab_y])
        count_yi_xi_xi1 = MultiDimnDict([self.vocab_y, self.vocab_x, self.vocab_x])
        for i , row in self.data.iterrows():
            x = row['x']
            y = row['y']
            _iter = 2
            while _iter < len(y) - 2:
                yi = y[_iter]
                xi = x[_iter]
                xi1 = x[_iter-1]
                count_yi_xi_xi1.data[yi][xi][xi1]+=1
                count_yi.data[yi]+=1
                _iter += 1
        for y in self.vocab_y:
            for x in self.vocab_x:
                yi_xi = 0
                for x1 in self.vocab_x:
                    try:
                        emissions.data[y][x][x1] += count_yi_xi_xi1.data[y][x][x1] / count_yi.data[y]
                    except:
                        print(y)
                        emissions.data[y][x][x1] = 0
                    yi_xi += emissions.data[y][x][x1]
                self.emissions_yi_xi.data[y][x] = yi_xi
        self.emissions = emissions
        return emissions


    def calculate_transitions(self):
        transitions = MultiDimnDict([self.vocab_y, self.vocab_y])
        count_yi1 = MultiDimnDict([self.vocab_y])
        count_yi1_yi = MultiDimnDict([self.vocab_y, self.vocab_y])
        for i , row in self.data.iterrows():
            y = row['y']
            _iter = 0
            while _iter < len(y) - 1:
                yi1 = y[_iter]
                yi = y[_iter+1]
                try:
                    count_yi1_yi.data[yi1][yi]+=1
                except:
                    print(yi1,yi)
                    print(count_yi1_yi)
                count_yi1.data[yi1]+=1
                _iter += 1
        for y in self.vocab_y:
            for y1 in self.vocab_y:
                try:
                    transitions.data[y][y1] += count_yi1_yi.data[y][y1] / count_yi1.data[y]
                except:
                    print(y)
                    transitions.data[y][y1] = 0

        self.transitions = transitions
        return transitions

    def viterbi(self, test_x):
        ## Final Goal: Find PI(n,v) where n is the length of y without start and stop
        ## We wont use PI we will maximize negative log pi
        ## Step 1: Find PI(1,v)
        temp_y = []
        _pi = []
        _y = []
        PI = []
        k = 0
        x = test_x[k]
        for y in self.vocab_y:
            a = self.emissions_yi_xi.data[y][x]
            b = self.transitions.data['START'][y]


            nlog_a = sys.maxsize 
            nlog_b = sys.maxsize 
            if a > pow(10,-7):
                nlog_a = -1 * math.log(a)
            if b > pow(10,-7):
                nlog_b = -1 * math.log(b)
            _pi.append(nlog_a + nlog_b)
            _y.append(y)
            
        min_pi = min(_pi)
        min_y = _y[_pi.index(min_pi)]

        yi1 = min_y
        PI.append(min_pi)
        temp_y.append(yi1)

        print("Done 1 ", _pi)

        k = 1
        while k < len(test_x):
            x = test_x[k]
            xi = test_x[k-1]
            _pi = []
            _y = []
            for y in self.vocab_y:
                a = self.emissions.data[y][x][xi]
                b = self.transitions.data[yi1][y]

                nlog_a = sys.maxsize 
                nlog_b = sys.maxsize 
                if a > pow(10,-7):
                    nlog_a = -1 * math.log(a)
                if b > pow(10,-7):
                    nlog_b = -1 * math.log(b)
                
                _pi.append(nlog_a + nlog_b + PI[-1])

                _y.append(y)
                

            min_pi = min(_pi)
            min_y = _y[_pi.index(min_pi)]

            yi1 = min_y
            PI.append(min_pi)
            temp_y.append(yi1)
            k += 1 

            if k % 100000 == 0:
                print("k = ",k,"    pi = ",PI[-5:])
        
        y_star = 'STOP'
        final_Y = []
        final_Y.append(y_star)


        for _PI in PI[::-1]:
            _pi = []
            _y = []
            for y in self.vocab_y:
                transition = sys.maxsize
                a = self.transitions.data[y][y_star]
                if a > pow(10,-7):
                    transition = math.log(a) * -1
                _pi.append(_PI + transition)
                _y.append(y)

            min_pi = min(_pi)
            final_y = _y[_pi.index(min_pi)]

            final_Y.append(final_y)
            y_star = final_y

        final_Y.append('START')
        return final_Y[::-1],temp_y

In [124]:
hmm = HMM(df, set(temp.iloc[0]['x']), set(temp.iloc[0]['y'] + ['START','STOP']))

In [125]:
start = timer()
hmm.calculate_emissions()
stop = timer()
print(stop-start)

START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
START
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
STOP
1.3960335369920358


In [126]:
start = timer()
hmm.calculate_transitions()
stop = timer()
print(stop-start)

STOP
STOP
STOP
STOP
STOP
1.2460597809986211


In [127]:
print(hmm.transitions)

N  
 {'N': 0.989996761607527, 'I': 0.0, 'E': 4.586958177032733e-06, 'START': 0.0, 'STOP': 0.009998651434295952}
I  
 {'N': 0.0, 'I': 0.9894096596110633, 'E': 0.0005687019724248898, 'START': 0.0, 'STOP': 0.010021638416511776}
E  
 {'N': 0.00013345789403443215, 'I': 0.0022154010409715734, 'E': 0.9876951821700254, 'START': 0.0, 'STOP': 0.009955958894968638}
START  
 {'N': 0.8570417551309271, 'I': 0.11354879295431312, 'E': 0.02940945191475977, 'START': 0.0, 'STOP': 0.0}
STOP  
 {'N': 0, 'I': 0, 'E': 0, 'START': 0, 'STOP': 0}



In [128]:
print(hmm.emissions)

N  
 G  
  {'G': 0.031661157651446434, 'N': 0.0, 'A': 0.04892585904597149, 'C': 0.03037774900795601, 'T': 0.058933825353452235}
 N  
  {'G': 0.0, 'N': 0.015594678768714036, 'A': 9.361113373380177e-07, 'C': 0.0, 'T': 0.0}
 A  
  {'G': 0.05064455946132409, 'N': 9.361113373380177e-07, 'A': 0.12019856793687614, 'C': 0.05926333654419522, 'T': 0.09191115554519592}
 C  
  {'G': 0.03577536697904702, 'N': 0.0, 'A': 0.05240070433017021, 'C': 0.032043091077080345, 'T': 0.05100402621486189}
 T  
  {'G': 0.05173138472397353, 'N': 0.0, 'A': 0.10048125483852548, 'C': 0.049503439741109045, 'T': 0.11954797055742622}
I  
 G  
  {'G': 0.032144601164905626, 'N': 0.0, 'A': 0.05075053609721229, 'C': 0.02949065456938832, 'T': 0.05854960049257249}
 N  
  {'G': 0.0, 'N': 0.0, 'A': 0.0, 'C': 0.0, 'T': 0.0}
 A  
  {'G': 0.050156052059816415, 'N': 0.0, 'A': 0.12540782312684448, 'C': 0.05715539388106073, 'T': 0.09785631886991415}
 C  
  {'G': 0.03528687393399812, 'N': 0.0, 'A': 0.052604760118613716, 'C': 0.0304673

In [129]:
start = timer()
y_pred,temp_y = hmm.viterbi(temp.iloc[4]['x'])
stop = timer()
print(stop-start)

Done 1  [1.2897605478845067, 3.279834123143049, 4.684233680101636, 18446744073709551614, 18446744073709551614]
k =  100000     pi =  [267954.6415362852, 267956.9493739801, 267959.3463604562, 267961.6541981511, 267964.05118462717]
k =  200000     pi =  [536708.6887466949, 536711.6604909289, 536714.5018846046, 536717.49486168, 536719.8026993748]
k =  300000     pi =  [804905.8336743325, 804907.9677655003, 804910.3647519763, 804912.6725896712, 804915.5139833469]
k =  400000     pi =  [1074866.3232219599, 1074869.3161990354, 1074871.4448628132, 1074874.4037516664, 1074877.8544789592]
k =  500000     pi =  [1345309.899159626, 1345312.033250794, 1345314.167341962, 1345316.5643284381, 1345318.872166133]
k =  600000     pi =  [1617866.7304760974, 1617869.5662941393, 1617871.8741318341, 1617874.8600361452, 1617877.8758028743]
k =  700000     pi =  [1888930.245078676, 1888932.3791698439, 1888934.5132610118, 1888936.6473521797, 1888939.044338656]
k =  800000     pi =  [2162498.9811460604, 2162501

In [130]:
print(len(y_pred),'\n',len(temp.iloc[4]['y']))
print(y_pred[-1], temp.iloc[4]['y'][-1])

1271728 
 1271726
STOP N


In [131]:
count = 0
g = ['START'] + temp.iloc[4]['y'] + ['STOP']
for i in range(len(y_pred)):
    if y_pred[i] == g[i]:
        count += 1
print(count/len(y_pred))

0.11338116326761698


In [132]:
print(set(y_pred))
print(set(g))
print(set(temp_y))
print([g.count(i) for i in set(g)])
print([temp_y.count(i) for i in set(temp_y)])
print([temp_y.count(i) for i in set(temp_y)])

{'STOP', 'I', 'START'}
{'N', 'I', 'E', 'START', 'STOP'}
{'N'}
[1090073, 144188, 37465, 1, 1]
[1271726]
[1271726]


In [84]:
temp_y


['N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N'